# Tutorial: Transaction Shielding with shield()

**Category**: Concurrency  
**Difficulty**: Intermediate  
**Time**: 20 minutes

## Overview

Database transactions require atomic commit/rollback - either all changes persist or none do. In async applications with cancellation, interrupting a commit corrupts data. lionherd-core's `shield()` creates a cancellation barrier around critical operations.

**What You'll Learn**:
- Use `shield()` to protect critical finalization
- Guarantee atomic commit/rollback under cancellation
- Detect cancellation with `is_cancelled()`
- Integrate shielding with context managers

**Prerequisites**:
```bash
pip install lionherd-core
```

In [1]:
# Standard library
from enum import Enum

# lionherd-core
from lionherd_core.libs.concurrency import fail_after, is_cancelled, shield, sleep

## Section 1: shield() Basics

**API Overview**:
```python
await shield(async_function)  # Protects function from cancellation
```

**What it does**:
- Creates cancellation barrier around async function
- Function runs to completion even if outer scope cancelled
- **Suppresses** outer cancellation completely (timeout ignored)
- Used in cleanup contexts where completion > deadline

**When to Use**:
- ✅ Cleanup in `__aexit__`, `finally` (exception already raised)
- ✅ Database commit/rollback (ACID guarantees)
- ✅ File write + fsync (partial writes corrupt data)
- ✅ Resource cleanup (must release locks, close connections)
- ❌ Normal operations (should respect cancellation)
- ❌ User-facing operations with timeouts (deadline ignored)

In [2]:
# Example 1: Basic shield usage
async def critical_operation():
    """Operation that must complete atomically."""
    print("[Critical] Starting...")
    await sleep(0.2)  # Simulate work
    print("[Critical] Completed successfully")
    return "done"


# Without shield - gets cancelled
print("Test 1: Without shield")
try:
    with fail_after(0.1):  # Timeout before completion
        await critical_operation()
except TimeoutError:
    print("[Test] Operation cancelled (incomplete!)\n")

# With shield - completes AND suppresses timeout
print("Test 2: With shield (suppression semantics)")
timeout_caught = False
result = None
try:
    with fail_after(0.1):  # Timeout fires at t=0.1s
        result = await shield(critical_operation)  # Completes at t=0.2s
        print(f"Result: {result}")  # THIS EXECUTES
except TimeoutError:
    timeout_caught = True
    print("[Test] TimeoutError caught")

print(f"[Test] Timeout suppressed: timeout_caught={timeout_caught}, result={result}")

Test 1: Without shield
[Critical] Starting...
[Test] Operation cancelled (incomplete!)

Test 2: With shield (suppression semantics)
[Critical] Starting...
[Critical] Completed successfully
Result: done
[Test] Timeout suppressed: timeout_caught=False, result=done


**Key Points**:
- **Suppression semantics**: `shield()` completely blocks outer cancellation
- **Timeout ignored**: After shielded work completes, no exception raised
- **Atomic guarantee**: Shielded function runs to completion or fails completely
- **Correct usage**: Use in cleanup contexts (`__aexit__`, `finally`) where exception already exists

**Important**: In bare usage (example above), the timeout was **suppressed**, not delayed. The `fail_after(0.1)` fired at t=0.1s, but shield blocked it. After completion at t=0.2s, no TimeoutError was raised.

**When this is correct**:
- In `__aexit__`: Exception already exists, shield ensures cleanup completes, `__aexit__` propagates exception
- Critical sections: Completion is more important than deadline (rare - use carefully)

## Section 2: Transaction Pattern

**Problem**: Database transactions need atomic commit/rollback. Cancellation during commit leaves database inconsistent.

**Solution**: Shield both commit and rollback operations.

In [3]:
# Example 2: Transaction with shielded finalization
class TransactionState(Enum):
    IDLE = "idle"
    ACTIVE = "active"
    COMMITTED = "committed"
    ROLLED_BACK = "rolled_back"


class Transaction:
    """Database transaction with atomic guarantees."""

    def __init__(self):
        self.state = TransactionState.IDLE
        self.operations = []

    async def begin(self):
        self.state = TransactionState.ACTIVE
        print("[Transaction] BEGIN")

    async def execute(self, sql: str):
        """Execute SQL (cancellable - normal work)."""
        if self.state != TransactionState.ACTIVE:
            raise RuntimeError(f"Cannot execute in state {self.state}")
        self.operations.append(sql)
        await sleep(0.01)
        print(f"[Transaction] Execute: {sql}")

    async def commit(self):
        """Commit transaction (MUST complete atomically)."""
        print("[Transaction] Committing...")
        await sleep(0.1)  # Simulate commit latency
        self.state = TransactionState.COMMITTED
        print("[Transaction] COMMIT completed")

    async def rollback(self):
        """Rollback transaction (MUST complete atomically)."""
        print("[Transaction] Rolling back...")
        await sleep(0.05)  # Simulate rollback latency
        self.state = TransactionState.ROLLED_BACK
        print("[Transaction] ROLLBACK completed")

    async def __aenter__(self):
        await self.begin()
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """Guarantee atomic finalization."""
        if exc_type is None:
            # Success - shield commit
            await shield(self.commit)
        else:
            # Error or cancellation - shield rollback
            if is_cancelled(exc_val):
                print("[Transaction] Cancellation detected, rolling back")
            await shield(self.rollback)

        return False  # Propagate exception after finalization


# Test 1: Successful transaction
print("Test 1: Success case")
async with Transaction() as txn:
    await txn.execute("INSERT INTO users VALUES (1, 'Alice')")
    await txn.execute("UPDATE accounts SET balance = 100")

print(f"Final state: {txn.state}\n")

# Test 2: Transaction with error
print("Test 2: Error triggers rollback")
try:
    async with Transaction() as txn:
        await txn.execute("INSERT INTO users VALUES (2, 'Bob')")
        raise ValueError("Constraint violation")
except ValueError as e:
    print(f"[Test] Caught: {e}")

print(f"Final state: {txn.state}\n")

Test 1: Success case
[Transaction] BEGIN
[Transaction] Execute: INSERT INTO users VALUES (1, 'Alice')
[Transaction] Execute: UPDATE accounts SET balance = 100
[Transaction] Committing...
[Transaction] COMMIT completed
Final state: TransactionState.COMMITTED

Test 2: Error triggers rollback
[Transaction] BEGIN
[Transaction] Execute: INSERT INTO users VALUES (2, 'Bob')
[Transaction] Rolling back...
[Transaction] ROLLBACK completed
[Test] Caught: Constraint violation
Final state: TransactionState.ROLLED_BACK



**Pattern**:
1. **Normal work is cancellable**: `execute()` can be interrupted
2. **Finalization is shielded**: Both `commit()` and `rollback()` protected
3. **Context manager**: `__aexit__` chooses commit or rollback
4. **Propagate after**: Exception delivered after cleanup completes

## Section 3: Cancellation Guarantees

**Critical Test**: Rollback must complete even when cancelled mid-transaction.

In [4]:
# Example 3: Cancellation during transaction
print("Test: Cancellation guarantees rollback")
txn_ref = None

try:
    with fail_after(0.08):  # Cancel before transaction completes
        async with Transaction() as txn:
            txn_ref = txn
            await txn.execute("INSERT INTO orders VALUES (1, 'pending')")
            print("[Test] Starting slow operation (will be cancelled)...")
            await sleep(1.0)  # This gets interrupted
            print("[Test] This won't print")
except TimeoutError:
    print("[Test] Timed out (expected)")

print(f"[Test] Final state: {txn_ref.state}")
print("[Test] ✅ Rollback completed despite cancellation")

Test: Cancellation guarantees rollback
[Transaction] BEGIN
[Transaction] Execute: INSERT INTO orders VALUES (1, 'pending')
[Test] Starting slow operation (will be cancelled)...
[Transaction] Cancellation detected, rolling back
[Transaction] Rolling back...
[Transaction] ROLLBACK completed
[Test] Timed out (expected)
[Test] Final state: TransactionState.ROLLED_BACK
[Test] ✅ Rollback completed despite cancellation


**What Happened**:
1. Transaction started, executed 1 query
2. Timeout fired during `sleep(1.0)` (cancellation requested)
3. Context manager `__aexit__` invoked with `TimeoutError`
4. `shield(self.rollback)` ran to completion despite cancellation
5. State transitioned to `ROLLED_BACK` (atomic guarantee)
6. `TimeoutError` propagated after cleanup completed

## Section 4: is_cancelled() for Debugging

`is_cancelled()` detects if an exception is cancellation-related (useful for logging).

In [5]:
# Example 4: Cancellation detection
class LoggingTransaction(Transaction):
    """Transaction with detailed cancellation logging."""

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        if exc_type is None:
            await shield(self.commit)
        else:
            # Distinguish cancellation from errors
            if is_cancelled(exc_val):
                print(f"[Transaction] CANCELLED ({type(exc_val).__name__}), rolling back")
            else:
                print(f"[Transaction] ERROR ({exc_val}), rolling back")

            await shield(self.rollback)

        return False


# Test cancellation vs error
print("Test 1: Cancellation")
try:
    with fail_after(0.05):
        async with LoggingTransaction() as txn:
            await txn.execute("INSERT ...")
            await sleep(1.0)
except TimeoutError:
    pass

print("\nTest 2: Regular error")
try:
    async with LoggingTransaction() as txn:
        await txn.execute("INSERT ...")
        raise ValueError("Invalid data")
except ValueError:
    pass

Test 1: Cancellation
[Transaction] BEGIN
[Transaction] Execute: INSERT ...
[Transaction] CANCELLED (CancelledError), rolling back
[Transaction] Rolling back...
[Transaction] ROLLBACK completed

Test 2: Regular error
[Transaction] BEGIN
[Transaction] Execute: INSERT ...
[Transaction] ERROR (Invalid data), rolling back
[Transaction] Rolling back...
[Transaction] ROLLBACK completed


**Use Cases for is_cancelled()**:
- **Logging**: Distinguish timeout/cancellation from errors in logs
- **Metrics**: Track cancellation rate separately from error rate
- **Alerting**: Don't alert on user-initiated cancellations
- **Cleanup**: Different cleanup strategies for cancellation vs errors

## Section 5: Production Considerations

**What to Shield**:
- ✅ Database commit/rollback
- ✅ File fsync operations
- ✅ Lock release
- ✅ Connection close
- ❌ Query execution (should be cancellable)
- ❌ API calls (should respect timeout)
- ❌ User-facing operations (timeout ignored is bad UX)

**Performance**:
- Overhead: Negligible compared to I/O operations
- No throughput impact

**Critical Understanding**:
- Shield **suppresses** outer cancellation (doesn't delay it)
- In cleanup contexts (`__aexit__`), original exception still propagates
- Shield ensures cleanup completes, `__aexit__` returning False propagates exception
- In bare usage, timeout is **ignored** - use carefully

**Error Handling**:
- Shield doesn't suppress *internal* exceptions (those raised inside shielded work)
- Rollback/commit failures propagate normally
- Use try/finally or context managers for guaranteed cleanup

In [6]:
# Example 5: Production-ready pattern (30 LOC)
class ProductionTransaction:
    """Minimal production transaction with shielding."""

    def __init__(self):
        self.state = "idle"

    async def begin(self):
        self.state = "active"
        # Your BEGIN logic here

    async def commit(self):
        await sleep(0.1)  # Your COMMIT logic
        self.state = "committed"

    async def rollback(self):
        await sleep(0.05)  # Your ROLLBACK logic
        self.state = "rolled_back"

    async def __aenter__(self):
        await self.begin()
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        # Shield both paths - atomic guarantee
        if exc_type is None:
            await shield(self.commit)
        else:
            await shield(self.rollback)
        return False


# Usage
async with ProductionTransaction() as txn:
    # Your database operations here
    await sleep(0.01)
    # Auto-commits (shielded) or rolls back on error (shielded)

print(f"✅ Transaction completed: {txn.state}")

✅ Transaction completed: committed


## Summary

### lionherd-core Shielding API

| Function | Purpose | Returns |
|----------|---------|----------|
| `shield(async_fn)` | Suppress outer cancellation | Awaitable result |
| `is_cancelled(exc)` | Detect cancellation | Boolean |

### Transaction Pattern

```python
async def __aexit__(self, exc_type, exc_val, exc_tb):
    if exc_type is None:
        await shield(self.commit)  # Success path
    else:
        await shield(self.rollback)  # Error/cancellation path
    return False  # Propagate exception after cleanup
```

**Why this works**: Exception already raised, shield ensures cleanup completes, `return False` propagates exception.

### Key Principles

1. **Shield suppresses outer cancellation**: Timeout/cancellation ignored while shielded work runs
2. **Shield both paths**: Commit AND rollback need protection
3. **Context managers work correctly**: Exception exists, shield ensures cleanup, `__aexit__` propagates
4. **Bare usage is footgun**: Timeout completely ignored - use only when completion > deadline
5. **Overhead is minimal**: Negligible vs I/O latency

### Suppression vs Delay

lionherd-core `shield()` uses **suppression semantics** (matches anyio/trio):
- Outer cancellation is **blocked**, not delayed
- After shielded work completes, no exception raised
- Correct for cleanup contexts (`__aexit__`, `finally`)
- **Caution**: In bare usage, timeouts are ignored

### When to Use shield()

- ✅ Database transactions (ACID guarantees in `__aexit__`)
- ✅ File operations requiring fsync (durability in cleanup)
- ✅ Resource cleanup (lock release, connection close)
- ✅ State transitions that must complete atomically
- ❌ Long-running computations (defeats cancellation purpose)
- ❌ User-facing operations (timeout ignored is bad UX)
- ❌ Retry loops (should be cancellable)

### When NOT to Use shield()

**Bare shield() usage can ignore timeouts:**
```python
# ⚠️ BAD: User expects 5s timeout
with fail_after(5.0):
    result = await shield(expensive_api_call)  # Takes 10s
    # Timeout ignored! User waits 10s
```

**Alternative: Use move_on_after for graceful degradation:**
```python
# ✅ GOOD: Timeout respected
with move_on_after(5.0) as scope:
    result = await expensive_api_call()
    if scope.cancelled_caught:
        result = fallback_value
```

### Related Resources

- [API Reference: Concurrency Utilities](../../docs/api/libs/concurrency/)
- [Tutorial: Resource Leak Detection](./)
- [anyio Cancellation Docs](https://anyio.readthedocs.io/en/stable/cancellation.html)